In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

current_time = datetime.now().strftime('%y-%m-%d')

# 로그인 정보 하드코딩
USERNAME = 'loextasy@naver.com'  # 여기에 Instagram 사용자 이름을 입력하세요
PASSWORD = 'v912803V!!!'  # 여기에 Instagram 비밀번호를 입력하세요

# 검색어 입력
search = input('검색어를 입력하세요: ')
search_input = search
search = urllib.parse.quote(search)
search_url = 'https://www.instagram.com/explore/tags/' + str(search) + '/'

# Chrome 드라이버 경로 설정
chrome_driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
service = Service(chrome_driver_path)

# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-notifications")  # 알림 비활성화
options.add_argument("--start-maximized")  # 브라우저 창 최대화

# WebDriver 객체 생성
driver = webdriver.Chrome(service=service, options=options)

# Instagram 로그인 페이지로 이동
driver.get('https://www.instagram.com/accounts/login/')
time.sleep(2)  # 페이지 로드 대기

# 로그인 과정
username_input = driver.find_element(By.NAME, 'username')
password_input = driver.find_element(By.NAME, 'password')
login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')

username_input.send_keys(USERNAME)
password_input.send_keys(PASSWORD)
login_button.click()

time.sleep(5)  # 로그인 후 대기

# 로그인 후 탐색
driver.get(search_url)
time.sleep(30)  # 페이지가 완전히 로드될 때까지 대기

# 스크롤하여 데이터를 수집하는 부분
SCROLL_PAUSE_TIME = 1
alt_texts = []  # alt 텍스트를 저장할 리스트

last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    # 페이지 소스 가져오기
    pagestring = driver.page_source
    bs = BeautifulSoup(pagestring, 'lxml')
    
    # _aagv 클래스를 가진 모든 div 요소 찾기
    post_divs = bs.find_all('div', {'class': '_aagv'})
    
    for div in post_divs:
        img_tags = div.find_all('img')
        for img in img_tags:
            alt_text = img.get('alt')
            if alt_text:
                alt_texts.append(alt_text)
    
    if post_divs:
        print(f"Found {len(post_divs)} divs with class '_aagv' on this scroll.")
    
    # 페이지 스크롤
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script('return document.body.scrollHeight')
    
    # 스크롤이 끝났는지 확인
    if new_height == last_height:
        break
    last_height = new_height

# 결과 출력
if alt_texts:
    data = pd.DataFrame({
        "검색어": search_input,
        "검색결과": alt_texts
    })
    print(data)
else:
    print("No alt text found.")

data.to_excel(f"인스타그램크롤링 {current_time}.xlsx", index=False)

# 드라이버 종료
driver.quit()

검색어를 입력하세요:  성수역


Found 12 divs with class '_aagv' on this scroll.
    검색어                                               검색결과
0   성수역  Photo by 에이치아이피 HIP on August 11, 2024. May be...
1   성수역  💍 팝업이 지겨울 때 가볼만 한 성수동 건축 여행지 7곳\n\n팝업 스토어로 늘 붐...
2   성수역  "이번 역은 '올리브영' 역입니다"... 2호선 핫플 성수역, 이름 교체된다\n\n...
3   성수역  ⚪️ X 11 \n클팍 회색 인생🙄\n그래도 할당량은 다 채울만큼 채운거 같다\n연...
4   성수역  Photo shared by 성수맛집 건대맛집 광진구맛집 동대문맛집 중랑구맛집 성동...
5   성수역  Photo shared by 서울여행 공식인스타 / 서울맛집 서울핫플 서울가볼만한곳...
6   성수역  서울교통공사(@seoulmetro.official)가 안전사고 우려가 제기된 지하철...
7   성수역  🐽 332번째 메뉴 : 성수역 아카사카 크라부\n\n성수역 데이트, 소개팅 100%...
8   성수역  Photo by 서울여행 공식인스타 / 서울맛집 서울핫플 서울가볼만한곳 in 성수역...
9   성수역  Photo by WIKITREE 위키트리 on August 11, 2024....
10  성수역  Photo by 오마_스페이스 on August 11, 2024. May be an...
11  성수역  성수동에 국내 최대 규모 올리브영 생긴대 ! !\n팩토리얼 서울 건물 1~5층을 사...
